In [4]:
# Add directory above current directory to path
import sys; sys.path.insert(0, '../..'); sys.path.insert(0, '..')

from functions.miscellanea import _unpickle

In [5]:
import torch
import numpy as np
from torch.distributions.multivariate_normal import MultivariateNormal as MVN
from functions.kernels import SumIdentical, RBF, RQ
from dependence.hsic import HSIC
from functions.miscellanea import ruled_print, _pickle, _unpickle
import argparse, os

device = torch.device('cuda')

save, reps, sizepow = 0,1,4

In [7]:
def numpy(tensor):
    return tensor.cpu().numpy()

def two_gaussians(n, covariance=[1,0,0,1]):
    sampler = MVN(loc=torch.zeros(2),
                  covariance_matrix=torch.Tensor([covariance[0:2],covariance[2:4]]))
    X,Y = sampler.sample((n,)).t()
    return X.view(-1,1), Y.view(-1,1)

def hsic_test(n, reps=1,
            kernels_X=None, params_X=None,
            kernels_Y=None, params_Y=None,
            save=False, cov=[1,0,0,1]):
    ''' samples X,Y from a 2d-gaussian with various levels of correlation '''
    hsic_u = HSIC(n=n,
                  kernels_X=kernels_X, params_X=params_X,
                  kernels_Y=kernels_Y, params_Y=params_Y,
                  unbiased=True)
    hsic_b = HSIC(n=n,
                  kernels_X=kernels_X, params_X=params_X,
                  kernels_Y=kernels_Y, params_Y=params_Y,
                  unbiased=False)
    experiments = []
    for _ in range(reps):
        X,Y = two_gaussians(n, covariance=cov)
        X,Y = X.to(device), Y.to(device)
        curr_experiment = [hsic_b(X,Y), hsic_u(X,Y), *hsic_b.GammaProb(X,Y, alpha=0.05)]
        curr_experiment = [numpy(val) if isinstance(val,torch.Tensor) else val.ravel()[0] for val in curr_experiment]
        curr_experiment = np.array(curr_experiment)
        experiments.append(curr_experiment)

    del hsic_b, hsic_u
    torch.cuda.empty_cache()
    return experiments

In [9]:

n = 10 ** sizepow


params_X = [{'bandwidth':10**(i),
            'alpha':(4+i)
            } for i in range(-2,2)]
params_Y = params_X
kernels_X = [RQ for p in params_X]
kernels_Y = [RBF for p in params_Y]
#print(kernels,params)
if save:
    save = {'params':[], 'test_values':[]}

In [18]:
eps += 1/25
print('\n\n')
ruled_print(f'Testing MVN(0,cov) with cov=[1,{eps};{eps},1] with rho={eps} increasing')
cov = [1,eps,eps,1]
results = hsic_test(n, reps=reps,
                    kernels_X=kernels_X, params_X=params_X,
                    kernels_Y=kernels_Y, params_Y=params_Y,
                    save=save, cov=cov)

hsic_b, hsic_u, gamma_stat, gamma_test_thresh = results[0]
print(f'Biased HSIC : {hsic_b} | versus Unbiased HSIC (possibly negative) : {hsic_u}')
ruled_print(f'Gamma Approximation to the H0 distribution at alpha = 0.05 : is {gamma_stat} > {gamma_test_thresh} ?')




--------------------------------------------------------------------
Testing MVN(0,cov) with cov=[1,0.12;0.12,1] with rho=0.12 increasing
--------------------------------------------------------------------


RuntimeError: CUDA out of memory. Tried to allocate 382.00 MiB (GPU 0; 5.80 GiB total capacity; 4.48 GiB already allocated; 184.00 MiB free; 1.86 MiB cached)

In [17]:
%whos

Variable            Type        Data/Info
-----------------------------------------
HSIC                type        <class 'dependence.hsic.HSIC'>
MVN                 type        <class 'torch.distributio<...>rmal.MultivariateNormal'>
RBF                 type        <class 'functions.kernels.RBF'>
RQ                  type        <class 'functions.kernels.RQ'>
SumIdentical        type        <class 'functions.kernels.SumIdentical'>
argparse            module      <module 'argparse' from '<...>b/python3.6/argparse.py'>
cov                 list        n=4
device              device      cuda
eps                 float       0.08
gamma_stat          float64     14.330674171447754
gamma_test_thresh   float64     317.8238447547165
hsic_b              float64     0.0014330658596009016
hsic_test           function    <function hsic_test at 0x7fa37c35c0d0>
hsic_u              float64     3.8010644912719727
kernels_X           list        n=4
kernels_Y           list        n=4
main              